<a href="https://colab.research.google.com/github/RiseAboveAll/Model_Deployment/blob/master/Model_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Deployment 

<h1> Agenda </h1>

- When we develop a model in jupyter notebook , model is in the RAM of the system, the moment we close jupyter notebook model is gone , hence we need to retrain which is not viable. Hence we need to save model in the disk. It is called **Serialization of Machine Learning Model**.

- Productionising model in real time . In production one record come in at a time . We expose mode as Rest API.

- We give model to testing team , they might say model does not run their . Packaging the model for reproducability . Docker sets virtual enviornment, where along with your model, dependencies for model will also be packaged, we just give files which defines the particular environment. 

- Pipeline

- Scaling - Instead of one request , what if 100 request come at same time 

# Model Deployment

- Putting the model in server , enabling it to take input data inorder to return an output is model deployment

- Ensure data from multiple system .

- Fetching records from outer source

When we deploy model we can have : 

1. Batch Mode for Deploying Model : 

  We store all information in database and run the model till that point again. Hence we score for all the record till the latest being entered in the record. 

2. Real Time for Deploying Model :

  Instant response for the data you gave to model. It require infrastructure for responding milisecond. 

You need to save your model , the process is called **Pickeling / Serialization**



# Model Serialization - Pickling

library used to save model is pickle, **pickle.dump(model-name)** is the command to save model onto the disk. **pickle.load** is the command to load a model from the disk on to the memory. To score the new record you would use pickle.load() , it will load the model and then predict the score .


```
import os
import pickle
filename='himanshu.pras'
#It store the model in the given filename and write the content of the model in write binary mode.

pickle.dump(model,open(filename,'wb')) 

# Load Model in read binary mode :
loaded_model=pickle.load(open(filename,'rb'))
result=loaded_model.predict(X_test)

```

Alternate Method :

```
from sklearn.externals import jobilb
filename='himanshu.sav'
joblib.dump(model,filename)
loaded_model=joblib.load(filename)
result=loaded_model.predict(X_test)
```


# Updatable Classifiers

- To make model continuously learn . It is based on incremental learning .

- We split dataset in multiple parts and keep training model incrementaly, chunk by chunk. Model will continuously update weights as per the chunk.

- This can be used for streaming data.

- Incremental classifiers are :

  - MultinomialNB

  - BernoulliNB

  - Perceptron

  - SGDClassifier

<h1>Incremental Classifier</h1>

```
from sklearn import linear_model
chunksize=25
for chunk in pd.read_csv('data.csv',header=None,chunksize=chunksize):
  train_sub=chunk
  y=train_sub.iloc[:,4]
  X=train_sub.drop([4],axis=1)
  clf=linear_model.SGDClassifier()
  clf.partial_fit(X,y,classes=np.unique(y))
  pred=clf.predict(test)
  print(accuracy_score(pred,labeld_test))

```

# Batch Mode

- We have task schedulers , we pick the script of python and run at given particular time. In this we load the python scrit while loading the model via pickle file. 

- We have a source data , we process the data and then make it available to the model whose pickle file is loaded. Model will make prediction for new records and print it in database .






# Real Time Prediction 

<img src=https://raw.githubusercontent.com/RiseAboveAll/Model_Deployment/master/Real_Time_Mode.PNG>


- ReactJS is the front end, it sends request to server.  

- Backend APIs are provided as Micro-Services. It consumes data from database also. On the Server end it processes the request and respond to ReactJS.

- ReactJS is the client, and rest are the server components which process your data received from the client. 

- As soon as the submit button is clicked on the client end, you will get the real time prediction. As client will be able to send the data to the server where the model is loaded or exposed and client will get response immediately. 

- Things to think about : 

  - How many servers to manage?

  - How to manage the workload?

- In server end your model runs as Rest API. It keeps listening and waiting for any requests that comes from the client. 

<h1>Server :</h1>

Steps :

1. Go to Command Prompt

2. Go to Directory where your server script is stored , use : cd path

3. You will find server.py file in the directory 

4. Run 'python server.py' command

<h1>Client :</h1>

Steps :

1. Go to Command Prompt

2. Go to Directory where your client script is stored , use : cd path

3. You will find client.py file in the directory 

4. Run 'python client.py' command






<h1>server.py</h1>

```
import numpy as np
import os
import pickle 
from flask import Flask,request,jsonify

os.chdir(path-where-pickle-is)
#Load the model
model=pickle.load(open('model-filename.sav','rb'))
app=Flask(__name__)
@app.route('/api',methods=['POST'])
def predict():
  #Get data from POST request
  data=request.get_json(force=True)
  # Make Prediction using model loaded as per the data
  predict_request=[[data['sl'],data['sw'],data['pl'],data['pw']]
  predict_request=np.array(predict_request)
  prediction=model.predict(predict_request)
  #Take the first value of prediction
  output=prediction[0]
  print(output)
  return jsonify(int(output))

if __name__=='__main__':
  app.run(port=8011,debug=True)

```



<h1>client.py</h1>

```
import requests
import json
url='http://localhost:8011/api'
data=json.dumps({'sl':3,'sw':2,'pl':1,'pw':5})
r=requests.post(url,data)
```

# Docker Concentration - Development Enviornment 

Each model have its own different dependencies. Some can be using pandas different version. 

Using Docker :

- There will be no enviroment issues

- No OS issues

- Preconfigured Environment

- Build once and run anywhere with docker


Like Github we also have Dockerhub hub.docker.com.. Dockerhub is something where lot of image is available (model image). 

Steps : 

1. Create a docker folder

2. In folder you need to have a docker file and docker-requirements.txt

3. Docker file consist of commands to be executed in docker 

```
FROM python:3
RUN apt-get update && apt-get install -y python3-pip
COPY requirements.txt
RUN pip install -r requirements.txt
#Install Jupyter
RUN pip3 install jupyter
#Create a new system user
RUN useradd -ms /bin/bash demo
#Change to this new user
USER demo
#Set container working directory to the user home folder
WORKDIR /home/demo
#Start Jupyter Notebook
ENTRYPOINT ["jupyter","notebook","--ip=0.0.0.0"]
```
In requirements.txt file it contains the library and their versions

To check docker Images : docker images 

To create Docker :

- Go to command prompt, change your directory to the disk where your docker file and requirements.txt is located

- To create new image of python along with the requred libraries : write command : docker build -t <name-of-image> . (This dot(.) means that files are located in the current directory.

- docker images

- To run the image : docker run -it <name-of-image>

- This won't run jupyter notebook in interactive session we need to give another command for it 

- To shut down docker press cntrl+c 


- To run jupyter notebook : docker run -p <port-number : 8888> <name-of-image>





# Docker Concentration - Productionization



<h1>Docker File</h1>

```
#Install the Image which has Python 3.6 , Flask and nginx server installed

> FROM tiangolo/uwsgi-nginx-flask:python3.6

>WORKDIR /app/

>COPY requirements.txt /app/

>RUN pip install -r ./requirements.txt 

#Set the environment name as production

>ENV ENVIRONMENT production

#Execute the App, i.e execute a server

>COPY main.py __init__.py /app/
```
<h1>requirements.txt</h1>

```
numpy==1.13
scipy==0.19.1
Flask==0.12.2
pandas==0.20.2
scikit_learn==0.18.2
```



- You need to create empty __init__.py file .

- You need to create main.py file which is the server file

<h1>main.py</h1>

```
import os
from flask import Flask,request
import pandas as pd
from sklearn import linear_model
from sklearn import datasets
import pickle
import numpy as np
import sklearn
from sklearn import ensemble 
from sklearn import model_selection

iris=datasets.load_iris()
train=iris.data
labels=iris.target

#creating and saving model
df Ensemble():
  rf = ensemble.RandomForestClassifier(n_estimators=500)
  rf.fit(train,label)
  pickle.dump(rf,open('iris.pkl','wb')

Ensemble()

app=Flask(__name__)
@app.route('/isAlive')

def index():
  return "true"

@app.route('/predict', methods=['POST'])
def get_prediction():
  #Works only for single sample
  data=request.get_json() #Get Data posted As Json
  data=np.array(data)[np.newaxis,:] #Converts shape from (4,) to (1,4)
  model = pickle.load(open('iris.pkl','rb'))
  prediction=model.predict(data)#runs globally loaded model on the data
  return str(prediction[0])

if __name__ == '__main__':
  if os.environ['ENVIRONMENT']=='production' :
    app.run(port=80,host='0.0.0.0')

```

Now since all the files are ready , we need to build a docker image. Steps are as follows :

- Change your path to these file located disk : cd path

- Build image using the files in the path : docker build -t <image-name> . 

- Map port in Docker & Run Server : docker run -p 3456:80 <image-name>

- Open new CMD window : curl -X POST 0.0.0.0:3456/predict -H 'Content-Type: application/json' -d '[8,7,7,6]'





# Kubernetes

- It is about how to productionalize your model on cloud

- Used for dealing with the situation when lot many request are coming , hence kubernetes cluster auto cluster your models. We port docker image on to kubernetes cluster which auto scale your nodes based on workload that is coming.

- It will provide the load balancer inorder to distribute the load  across the nodes.

- It also take care of fault tolerence